# Study question 1.5.3

Given the graph

$$
X_{1} \rightarrow X_{2} \rightarrow X_{3} \rightarrow X_{4}
$$

The joint probability distribution can be decomposed as

$$
P(X_{1}, X_{2}, X_{3}, X_{4}) = P(X_{1}) P(X_{2} | X_{1}) P(X_{3} | X_{2}) P(X_{4} | X_{3})
$$

The question gives us the following probabilities

$$
P(X_{i} = 1 \mid X_{i-1} = 1) = p\\
P(X_{i} = 1 \mid X_{i-1} = 0) = q\\
P(X_{i} = 1) = p_{0}
$$

We can use all of this to set up a little simulation to check our answers as we're going along.

In [7]:
import numpy as np

def simulate_graph(p, q, p0, n):
    X1 = (np.random.uniform(size = n) < p0) * 1

    X2 = np.zeros_like(X1)
    X2[X1 == 1] = (np.random.uniform(size = np.sum(X1 == 1)) < p) * 1
    X2[X1 == 0] = (np.random.uniform(size = np.sum(X1 == 0)) < q) * 1

    X3 = np.zeros_like(X2)
    X3[X2 == 1] = (np.random.uniform(size = np.sum(X2 == 1)) < p) * 1
    X3[X2 == 0] = (np.random.uniform(size = np.sum(X2 == 0)) < q) * 1

    X4 = np.zeros_like(X3)
    X4[X3 == 1] = (np.random.uniform(size = np.sum(X3 == 1)) < p) * 1
    X4[X3 == 0] = (np.random.uniform(size = np.sum(X3 == 0)) < q) * 1

    return (X1, X2, X3, X4)

1) Compute $P(X_{1} = 1, X_{2} = 0, X_{3} = 1, X_{4} = 0)$

Using the joint probability distribution together with the probabilities provided directly

$$
P(X_{1} = 1, X_{2} = 0, X_{3} = 1, X_{4} = 0) = p_0 (1-p) q (1-p)
$$

In [18]:
# Pick some random values to check our answer
p = 0.74
q = 0.23
p0 = 0.44
n = 1000000

X1, X2, X3, X4 = simulate_graph(p, q, p0, n)

mask = np.logical_and(X1 == 1, X2 == 0)
mask = np.logical_and(mask, X3 == 1)
mask = np.logical_and(mask, X4 == 0)
simulated_prob = np.sum(mask) / n

calculated_prob = p0*(1-p)*q*(1-p)

print(simulated_prob, calculated_prob)

0.00683 0.006841120000000001


2) Compute $P(X_{4} = 1 \mid X_1 = 1)$

We can do this in stages

$$
p(X_{2} = 1 \mid X_{1} = 1) = p\\
$$

$$
\begin{align}
P(X_{3} = 1 \mid X_{1} = 1) =\ &P(X_{3} = 1 \mid X_{2} = 1)P(X_{2} = 1 \mid X_{1} = 1)\ +\\
&P(X_{3} = 1 \mid X_{2} = 0)P(X_{2} = 0 \mid X_{1} = 1)\\
=\ &p^{2} + q(1-p)
\end{align}
$$

$$
\begin{align}
P(X_{4} = 1 \mid X_{1} = 1) =\ &P(X_{4} = 1 \mid X_{3} = 1)P(X_{3} = 1 \mid X_{1} = 1)\ +\\
&P(X_{4} = 1 \mid X_{3} = 0)P(X_{3} = 0 \mid X_{1} = 1)\\
=\ &p(p^{2} + q(1-p)) + q(1 - p^{2} + q(1-p))\\
=\ &p^{3} - qp^{2} - (1-p)q^{2} + (1-p)qp + q
\end{align}
$$

In [20]:
# Pick some random values to check our answer
p = 0.34
q = 0.46
p0 = 0.24
n = 1000000

X1, X2, X3, X4 = simulate_graph(p, q, p0, n)

simulated_prob = np.mean(X4[X1 == 1])
calculated_prob = p**3 - q*p**2 - (1-p)*q**2 + (1-p)*q*p + q

print(simulated_prob, calculated_prob)

0.4092441562446605 0.409696


3) Compute $P(X_{1} = 1 \mid X_4 = 1)$

This is our result from the last question swapped around. Bayes theorem can help us out with this.

$$
P(X_{1} = 1 \mid X_4 = 1) = \frac{
P(X_{4} = 1 \mid X_1 = 1)P(X_{1} = 1)
}{
P(X_{4} = 1)
}
$$

The missing piece for this is $P(X_{4} = 1)$, which we can calculate in stages again

$$
P(X_{2} = 1) = pp_{0} + q(1-p_{0})\\
$$

$$
\begin{align}
P(X_{3} = 1) &= pP(X_{2} = 1) + qP(X_{2} = 0)\\
&= p(pp_{0} + q(1-p_{0})) + q(1 - pp_{0} + q(1-p_{0}))\\
&= p_{0}p^{2} + (p_{0}-1)*q^{2} + (1-2p_{0})pq + q
\end{align}
$$

$$
\begin{align}
P(X_{4} = 1) &= pP(X_{3} = 1) + qP(X_{3} = 0)\\
&= p(p_{0}p^{2} + (p_{0}-1)*q^{2} + (1-2p_{0})pq + q) + q(1 - p_{0}p^{2} + (p_{0}-1)*q^{2} + (1-2p_{0})pq + q)\\
&= p_{0}p^{3} - (p_{0}-1)q^{3} + (1-3p_{0})p^{2}q + (3p_{0}-2)pq^{2} - q^{2} + pq + q
\end{align}
$$

Therefore...

$$
P(X_{1} = 1 \mid X_4 = 1) = \frac{
(p^{3} - qp^{2} - (1-p)q^{2} + (1-p)qp + q)p_{0}
}{
p_{0}p^{3} - (p_{0}-1)q^{3} + (1-3p_{0})p^{2}q + (3p_{0}-2)pq^{2} - q^{2} + pq + q
}
$$

Yeesh!



In [24]:
# Pick some random values to check our answer
p = 0.12
q = 0.53
p0 = 0.78
n = 1000000

X1, X2, X3, X4 = simulate_graph(p, q, p0, n)

simulated_prob = np.mean(X1[X4 == 1])
numerator = (p**3 - q*p**2 - (1-p)*q**2 + (1-p)*q*p + q) * p0
denominator = p0*p**3 - (p0-1)*q**3 + (1-3*p0)*q*p**2 + (3*p0 - 2)*p*q**2 - q**2 + p*q + q
calculated_prob = numerator / denominator

print(simulated_prob, calculated_prob)

0.746304625401171 0.7460181978448007


4) Compute $P(X_{3} = 1 \mid X_{1} = 0, X_{4} = 1)$

Again the conditional version of Bayes theorem is our friend for this one

$$
P(X_{3} = 1 \mid X_{1} = 0, X_{4} = 1) = \frac{
P(X_{4} = 1 \mid X_{3} = 1, X_{1} = 0)P(X_{3} = 1 \mid X_{1} = 0)
}{
P(X_{4} = 1 \mid X_{1} = 0)
}
$$

Since $X_{4}$ is dependent only on the value of $X_{3}$, we have

$$
P(X_{4} = 1 \mid X_{3} = 1, X_{1} = 0) = p
$$

That leaves a couple of other missing pieces to calculate

$$
\begin{align}
P(X_{3} = 1 | X_{1} = 0) =\ &P(X_{3} = 1 | X_{2} = 1)P(X_{2} = 1 | X_{1} = 0)\ +\\
&P(X_{3} = 1 | X_{2} = 0)P(X_{2} = 0 | X_{1} = 0)\\
=\ &pq + q(1-q)\\
=\ &pq + q - q^{2}
\end{align}
$$

$$
\begin{align}
P(X_{4} = 1 | X_{1} = 0) =\ &P(X_{4} = 1 | X_{3} = 1)P(X_{3} = 1 | X_{1} = 0)\ +\\
&P(X_{4} = 1 | X_{3} = 0)P(X_{3} = 0 | X_{1} = 0)\\
=\ &p(pq + q - q^{2}) + q(1-pq + q - q^{2})\\
=\ &q^{3} + p^{2}q - 2pq^{2} - q^{2} + pq + q
\end{align}
$$

Putting it all together...
$$
\begin{align}
P(X_{3} = 1 \mid X_{1} = 0, X_{4} = 1) &=
\frac{p(pq + q - q^{2})}{
q^{3} + p^{2}q - 2pq^{2} - q^{2} + pq + q
}\\
&=\frac{
p^2 + p - pq
}{
q^{2} + p^{2} - 2pq - q + p + 1
}\\
\end{align}
$$

In [ ]:
# Pick some random values to check our answer
p = 0.82
q = 0.23
p0 = 0.54
n = 10000000

X1, X2, X3, X4 = simulate_graph(p, q, p0, n)

simulated_prob = np.mean(X3[np.logical_and(X1 == 0, X4 == 1)])
numerator = p**2 + p - p*q
denominator = q**2 + p**2 - 2*p*q - q + p + 1
calculated_prob = numerator / denominator

print(simulated_prob, calculated_prob)